<a href="https://colab.research.google.com/github/Huyu2239/Prog2kakushin/blob/work1/work1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 評価の際のお願い
これは対話式アプリケーションなので、入力が必要です。
コードが崩れないように、入力フォームにカーソルが移ってからエンターを押してください。  
評価のためにデフォルトの値をそれぞれ指定しているので何も入力しなくても大丈夫です。  
念の為、コメントアウトにて入力例を示しています。



# 第12回(12/22) pandas
- 指定された任意の`csv`ファイルをダウンロード
- `DataFrame`に変換
- 入力されたコマンドどおりに`DataFrame`を処理


## 初期設定

In [29]:
# utils


class Sort():
    UP = True
    DOWN = False


def parse_command(raw_command):
    command_list = raw_command.split()
    return command_list[0], command_list


def sort_command(func):
    def wrapper(args):
        mode = Sort.DOWN
        raw_command = " ".join(args).split()
        main_command = raw_command[0]

        for arg in raw_command[1:]:
            subcommand = arg.split("=")[0]
            if subcommand == "key":
                key = arg[4:]
                if key not in df.columns.values:
                    raise RuntimeError("不明な引数")
            if subcommand == "mode":
                if arg[5:] in ["up", "UP", "Up"]:
                    mode = Sort.UP
                elif arg[5:] in ["down", "DOWN", "Down"]:
                    mode = Sort.DOWN
                else:
                    raise RuntimeError("不明な引数")
        func(key=key, mode=mode)
    return wrapper


def view_command(func):
    def wrapper(args):
        keys = df.columns.values
        raw_command = " ".join(args).split()
        main_command = raw_command[0]

        for arg in raw_command[1:]:
            subcommand = arg.split("=")[0]
            if subcommand == "keys":
                keys = arg[5:].split(",")
        func(keys=keys)
    return wrapper

In [30]:
# basic command
def echo(args):
    print(" ".join(args[1:]))


def help(args):
    print(
    "- 基本コマンド\n"
    "    help  : この文章を表示\n"
    "    echo  : 渡された文字列を表示\n"
    "- オリジナルコマンド\n"
    "    sort  : DataFrameを渡されたkey,modeを元にsort\n"
    "    view_values: DataFrameから合計値、平均値、最大値、最小値を計算し表示"
    )


In [31]:
# original command
@sort_command
def sort(key, mode):
    df.sort_values(key, ascending=bool(mode), inplace=True)

@view_command
def view_values(keys):
    for key in keys:
        print("\n")
        print(key)
        if df[key].dtype == "object":
            print("計算不能")
            continue
        print("合計値: ", df[key].sum())
        print("平均値: ", df[key].mean())
        print("最大値: ", df[key].max())
        print("最小値: ", df[key].min())



## メイン

csvファイルダウンロード

In [32]:
import os


file_url = input("ダウンロードするファイルのURLを入力してください: ") or "https://drive.google.com/uc?export=download&id=10vpfTNC2HnmfZGFQ6kc9HsuEuQ62yN1G"
file_name = input("ダウンロードしたファイルの名前を入力してください: ") or "csv_file.csv"

if not file_name.endswith(".csv"):
    file_name = "csv_file.csv"
    print("ファイル名が不適切なため、\"csv_file.csv\"に変更しました。")

os.system(f"wget \"{file_url}\" -O {file_name}")

# https://drive.google.com/uc?export=download&id=10vpfTNC2HnmfZGFQ6kc9HsuEuQ62yN1G
# https://drive.google.com/uc?export=download&id=1CDcR_0GFlx6R1tx-7IhyqE1g-HIcqhQj

ダウンロードするファイルのURLを入力してください: 
ダウンロードしたファイルの名前を入力してください: 


0

ダウンロードしたcsvをDataFrameに変換して表示  
ヘッダーが存在するかをユーザーに入力させる

In [33]:
import pandas as pd


df = pd.read_csv(file_name)
df

,選手名,年齢,身長,体重,出場数,ゴール数
0,川島永嗣,38,185,74,92,0
1,権田修一,32,187,83,22,0
2,シュミット・ダニエル,28,197,88,7,0
3,中村航輔,26,185,82,6,0
4,長友佑都,34,170,68,125,4
5,佐々木翔,31,177,70,11,0
6,谷口彰悟,29,183,75,5,0
7,昌子源,28,182,76,19,1
8,山根視来,27,178,72,3,1
9,室屋成,27,176,69,14,0


In [34]:
while True:
    header = input("headerは含まれますか？yes or no: ") or "yes"
    if header == "yes":
        break
    elif header == "no":
        df = pd.read_csv(file_name, header=None)
        break
    else:
        print("予期しない入力を受け付けました。もう一度入力してください。")
        continue
df

headerは含まれますか？yes or no: 


,選手名,年齢,身長,体重,出場数,ゴール数
0,川島永嗣,38,185,74,92,0
1,権田修一,32,187,83,22,0
2,シュミット・ダニエル,28,197,88,7,0
3,中村航輔,26,185,82,6,0
4,長友佑都,34,170,68,125,4
5,佐々木翔,31,177,70,11,0
6,谷口彰悟,29,183,75,5,0
7,昌子源,28,182,76,19,1
8,山根視来,27,178,72,3,1
9,室屋成,27,176,69,14,0


コマンド実行

In [35]:
def main():
    raw_command = input("$ ") or "view_values"
    if len(raw_command) == 0:
        return print("No command was entered.")
    main_command, args = parse_command(raw_command)
    try:
        eval(f"{main_command}({args})")
    except Exception as e:
        raise e


if __name__ == "__main__":
    main()

df

# sort key=出場数 mode=down
# view_values keys=年齢,体重,身長

$ view_values keys=年齢,体重,身長


年齢
合計値:  693
平均値:  27.72
最大値:  38
最小値:  24


体重
合計値:  1829
平均値:  73.16
最大値:  88
最小値:  63


身長
合計値:  4491
平均値:  179.64
最大値:  197
最小値:  170


,選手名,年齢,身長,体重,出場数,ゴール数
0,川島永嗣,38,185,74,92,0
1,権田修一,32,187,83,22,0
2,シュミット・ダニエル,28,197,88,7,0
3,中村航輔,26,185,82,6,0
4,長友佑都,34,170,68,125,4
5,佐々木翔,31,177,70,11,0
6,谷口彰悟,29,183,75,5,0
7,昌子源,28,182,76,19,1
8,山根視来,27,178,72,3,1
9,室屋成,27,176,69,14,0
